In [1]:
# install huggingface and datasets
!pip install -q datasets transformers
!pip install torch
!pip install pandas
!pip install pyarrow

     |████████████████████████████████| 325 kB 5.4 MB/s 
     |████████████████████████████████| 4.0 MB 42.6 MB/s 
     |████████████████████████████████| 1.1 MB 39.4 MB/s 
     |████████████████████████████████| 77 kB 5.1 MB/s 
     |████████████████████████████████| 136 kB 32.2 MB/s 
     |████████████████████████████████| 212 kB 49.9 MB/s 
     |████████████████████████████████| 127 kB 48.7 MB/s 
     |████████████████████████████████| 596 kB 47.0 MB/s 
     |████████████████████████████████| 895 kB 36.3 MB/s 
     |████████████████████████████████| 6.6 MB 28.1 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
     |████████████████████████████████| 144 kB 36.2 MB/s 
     |████████████████████████████████| 271 kB 49.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which i

In [2]:
from datasets import load_dataset, ClassLabel
from transformers import RobertaForTokenClassification, RobertaTokenizer, TrainingArguments, Trainer
import numpy as np
import torch
from sklearn import metrics


import pandas as pd
import datasets
from datasets.features import ClassLabel

In [3]:
torch.cuda.is_available()
device = torch.device("cuda")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /content/drive/MyDrive/e_ML4NLP2	

'cell to copy in working models.ipynb'	 model2_fr_newseye.ipynb
 hipe_code				 model2_sv_newseye.ipynb
 HIPE_experiement_Records.gsheet	 NER_Literature-reviews.gdoc
 model2_de_newseye.ipynb		 niclas_models
 model2_de_sonar.ipynb			 Outputs
 model2_en_topres19.ipynb		 presentation_papers
 model2_fi_newseye.ipynb		'Trial 1_ old-model-collection'
 model2_fr_hipe2020.ipynb		 v2.1
 model2_fr_letemps.ipynb


In [ ]:
datapath = '/content/drive/MyDrive/e_ML4NLP2/v2.1/'

In [ ]:
train_path = datapath + 'hipe2020/de/HIPE-2022-v2.1-hipe2020-train-de.tsv'
val_path = datapath + 'hipe2020/de/HIPE-2022-v2.1-hipe2020-dev-de.tsv'
test_path = datapath + 'hipe2020/de/HIPE-2022-v2.1-hipe2020-test_allmasked-de.tsv'

model = 'xlm-roberta-large-finetuned-conll03-german'

In [ ]:
# import dataset from cloned git repo
def load_dataset(path):
    df = pd.read_csv(path, sep='\t', skip_blank_lines=False, engine='python', quoting=3)
    # error_bad_lines=False, 
    return df


tsv_train = load_dataset(train_path)
# tsv_dev = load_dataset(dev_path)
tsv_test = load_dataset(test_path)

In [ ]:
def simple_preprocess(dataframe):
    # Add end_of_document token in df
    dataframe = dataframe.dropna(subset=['TOKEN'])

    # Filter out metadata rows beginning with #
    dataframe = dataframe[~dataframe['TOKEN'].astype(str).str.startswith('#')]
    dataframe = dataframe[~dataframe['TOKEN'].astype(str).str.startswith('\t')]

    #transforming nan var from Float to string to use in (***)
    dataframe.MISC = dataframe.MISC.fillna('')

    return dataframe

In [ ]:
tsv_train = simple_preprocess(tsv_train)
# tsv_dev = simple_preprocess(tsv_dev)
tsv_test = simple_preprocess(tsv_test)
#tsv_train = tsv_train.reset_index()
tsv_train.head(100)

,TOKEN,NE-COARSE-LIT,NE-COARSE-METO,NE-FINE-LIT,NE-FINE-METO,NE-FINE-COMP,NE-NESTED,NEL-LIT,NEL-METO,MISC
11,From,O,O,_,_,_,_,_,_,_
12,the,O,O,_,_,_,_,_,_,_
13,AMERICAN,B-prod,O,_,_,_,_,NIL,_,_
14,MERCURY,I-prod,O,_,_,_,_,NIL,_,NoSpaceAfter
15,.,O,O,_,_,_,_,_,_,EndOfLine|EndOfSentence
...,...,...,...,...,...,...,...,...,...,...
106,may,O,O,_,_,_,_,_,_,_
107,deduce,O,O,_,_,_,_,_,_,_
108,his,O,O,_,_,_,_,_,_,_
109,ar,O,O,_,_,_,_,_,_,NoSpaceAfter


In [ ]:
label_set = tsv_train['NE-COARSE-LIT'].unique()
# define the label mapping for NER
label_list = label_set.tolist()
label_list.append('_')
label_num = len(label_list)

labels = ClassLabel(num_classes=label_num, names=label_list)

labels

ClassLabel(num_classes=12, names=['O', 'B-prod', 'I-prod', 'B-loc', 'I-loc', 'B-org', 'I-org', 'B-time', 'I-time', 'B-pers', 'I-pers', '_'], id=None)

In [ ]:
def create_huggingface_file(dataframe):
    import pyarrow as pa
    import re
    from datasets import Dataset

    #creating dataset in json
    hug_out = []
    idx = 0 
    items = {'id': idx,'words':[ ], 'ner': [ ]}
    hug_out.append(items)
    for index, row in dataframe.iterrows():
        if  not re.search(r'EndOfSentence', row['MISC']):
            items['words'].append(row['TOKEN'])
            items['ner'].append(labels.str2int(row['NE-COARSE-LIT']))
        else:
            items['words'].append(row['TOKEN'])
            items['ner'].append(labels.str2int(row['NE-COARSE-LIT']))
            idx += 1
            items = {'id': idx,'words':[ ], 'ner': [ ]}
            hug_out.append(items)
    #filter hug_out out, delete items which has len(words) > 380
    #hug_out = filter(lambda x: len(x['words']) < 380, hug_out)
    #json to df
    hug_out = pd.DataFrame(hug_out)

    # delete all sentences that are too long
    #hug_out = hug_out[hug_out['words'].map(len) < 512] #why does not work? QA

    ### convert to Huggingface dataset
    hug_out = Dataset(pa.Table.from_pandas(hug_out))

    return hug_out

In [ ]:
train = create_huggingface_file(tsv_train)
# val = create_huggingface_file(tsv_dev)
test = create_huggingface_file(tsv_test)

#look at training data
for i in range(10):
  print(train[i])

print(len(train))
# print(len(val))
print(len(test))

{'id': 0, 'words': ['From', 'the', 'AMERICAN', 'MERCURY', '.'], 'ner': [0, 0, 1, 2, 0]}
{'id': 1, 'words': ['The', 'APOLOGIZER', '-', 'No', '.'], 'ner': [0, 0, 0, 0, 0]}
{'id': 2, 'words': ['II', '.'], 'ner': [0, 0]}
{'id': 3, 'words': ['A', 'S', 'my', 'enquiries', 'arc', 'extended', 'into', 'the', 'nature', 'of', 'anti', '-', 'federalifm', ',', 'and', 'of', 'the', 'motives', 'which', 'acftuate', 'fuch', 'people', ',', 'I', 'become', 'more', 'convinced', ',', 'that', 'my', 'deflgn', 'of', 'a', 'general', 'apology', 'for', 'them', 'is', 'very', 'meritorious', ',', 'and', 'ought', 'to', 'have', 'been', 'made', 'long', 'ago', ';', 'and', 'I', 'cannot', 'conceive', 'the', 'reafon', 'why', 'it', 'hath', 'never', 'been', 'publicly', 'attempted', ',', 'unlefs', 'it', 'be', 'the', 'excreme', 'difficulty', 'of', 'an', 'inveftigation', '.'], 'ner': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(model)

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/849 [00:00<?, ?B/s]

In [ ]:
train = train.filter(lambda example, idx: idx != len(train)-1, with_indices=True)
# val = val.filter(lambda example, idx: idx != len(val)-1, with_indices=True)
test = test.filter(lambda example, idx: idx != len(test)-1, with_indices=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
def tokenize_func(batch):
    tokenized = tokenizer(batch["words"], is_split_into_words=True, padding="max_length", max_length=100, truncation=True)
    tokenized_words = [
        [
            tokenizer(word, add_special_tokens=False)["input_ids"]
            for word in sent
        ]
        for sent in batch["words"]
    ]
    tokenized_ner = []

    for sent_words, sent_nes in zip(tokenized_words, batch["ner"]):
        tokenized_ner.append(
            [labels.str2int("O")]  # BOS symbol
            + [
                ne
                for subwords, ne in zip(sent_words, sent_nes)
                for _ in range(len(subwords))
            ]
            + [labels.str2int("O")]  # EOS symbol
        )

    # Padding with "O"
    tokenized["labels"] = [(ner + [labels.str2int("O")] * (100 - len(ner)))[:100] for ner in tokenized_ner]
    tokenized["subwords"] = tokenized_words
    return tokenized

def tokenize_nolabel_func(batch):
    tokenized = tokenizer(batch["words"], is_split_into_words=True, padding="max_length", max_length=100, truncation=True)
    tokenized_words = [
        [
            tokenizer(word, add_special_tokens=False)["input_ids"]
            for word in sent
        ]
        for sent in batch["words"]
    ]

    tokenized["subwords"] = tokenized_words
    return tokenized

#tiny_train_tokenized = tiny_train.map(tokenize_func, batched=True, batch_size=50)
#tiny_test_tokenized = tiny_test.map(tokenize_nolabel_func, batched=True, batch_size=50)

train_tokenized = train.map(tokenize_func, batched=True, batch_size=50)
# val_tokenized = val.map(tokenize_func, batched=True, batch_size=50)
test_tokenized = test.map(tokenize_nolabel_func, batched=True, batch_size=50)


  0%|          | 0/21 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [ ]:
model = RobertaForTokenClassification.from_pretrained(model, num_labels=labels.num_classes, ignore_mismatched_sizes=True)
# trainer = Trainer(model=model, args=training_args, train_dataset=train_tokenized)
trainer = Trainer(model=model, train_dataset=train_tokenized)
trainer.train()

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at Jean-Baptiste/roberta-large-ner-english and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([5, 1024]) in the checkpoint and torch.Size([12, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([12]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The following columns in the training set  don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: id, words, ner, subwords. If id, words, ner, subwords are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a fu

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=393, training_loss=0.06201068984946525, metrics={'train_runtime': 239.5591, 'train_samples_per_second': 13.087, 'train_steps_per_second': 1.641, 'total_flos': 568670924844000.0, 'train_loss': 0.06201068984946525, 'epoch': 3.0})

In [ ]:
predictions = trainer.predict(test_tokenized)

The following columns in the test set  don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: id, words, ner, subwords. If id, words, ner, subwords are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 553
  Batch size = 8


In [ ]:
outfile = open("/content/drive/MyDrive/e_ML4NLP2/Outputs/hipe2020_en/roberta-english-modern.tsv", "w")
outfile.write("TOKEN\tNE-COARSE-LIT\tNE-COARSE-METO\tNE-FINE-LIT\tNE-FINE-METO\tNE-FINE-COMP\tNE-NESTED\tNEL-LIT\tNEL-METO\tMISC")

for pred, tinput in zip(predictions.predictions, test_tokenized):
    pred_word_labels = []

    try:
        sent_len = tinput["input_ids"].index(0)
    except ValueError:
        sent_len = len(tinput["input_ids"])
    # Start at 1 to skip pred for cls token
    i = 1

    for word in tinput['subwords']:
        
        word_score = np.zeros_like(pred[0])
        eos_reached = False
        try:
            for subword in word:
                word_score += pred[i]
                i += 1
            label = labels.int2str(int(np.argmax(word_score)))
            pred_word_labels.append(label)
        except IndexError:
            pred_word_labels.append("O")

    for token, label in zip(tinput['words'], pred_word_labels):
        outfile.write("\n" + f"{token}\t{label}\t" + ('\t'.join(['_'] * 8)))

outfile.close()

In [ ]:
accuracy = datasets.load_metric("accuracy")
f1_metric = datasets.load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    refs = labels.flatten()
    predictions = np.argmax(logits, axis=-1)
    print(logits.shape)
    print(predictions.shape)
    pred = predictions.flatten()
    return {
        "accuracy": accuracy.compute(predictions=pred, references=refs)["accuracy"],
        "f1_micro": f1_metric.compute(predictions=pred, references=refs, average="micro")["f1"],
        "f1_macro": f1_metric.compute(predictions=pred, references=refs, average="macro")["f1"],
    }

In [ ]:
Trainer(
    model=model,
    args=training_args,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
).evaluate()

NameError: ignored